In [4]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
imdb_reviews=pd.read_csv("imdb_reviews.csv")
test_reviews=pd.read_csv("test_reviews.csv")
word_index=pd.read_csv("word_indexes.csv")
word_index=dict(zip(word_index.Words,word_index.Indexes))
word_index["<PAD>"]=0
word_index["<START"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3

In [7]:
def review_encoder(text):
  arr=[word_index[word] for word in text]
  return arr
train_data,train_labels=imdb_reviews['Reviews'],imdb_reviews['Sentiment']
test_data, test_labels=test_reviews['Reviews'],test_reviews['Sentiment']

In [8]:
train_data=train_data.apply(lambda review:review.split())
test_data=test_data.apply(lambda review:review.split())
train_data=train_data.apply(review_encoder)
test_data=test_data.apply(review_encoder)

In [9]:
def encode_sentiments(x):
  if x=='positive':
    return 1
  else:
    return 0

train_labels=train_labels.apply(encode_sentiments)
test_labels=test_labels.apply(encode_sentiments)
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=500)

In [10]:
model=keras.Sequential([keras.layers.Embedding(10000,16,input_length=500),
                        keras.layers.GlobalAveragePooling1D(),
                        keras.layers.Dense(16,activation='relu'),
                        keras.layers.Dense(1,activation='sigmoid')])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [11]:
#training the model
history=model.fit(train_data,train_labels,epochs=30,batch_size=512,validation_data=(test_data,test_labels))


Epoch 1/30
49/49 [==============================] - 3s 43ms/step - loss: 0.6918 - accuracy: 0.5475 - val_loss: 0.6893 - val_accuracy: 0.7325
Epoch 2/30
49/49 [==============================] - 2s 37ms/step - loss: 0.6830 - accuracy: 0.6956 - val_loss: 0.6756 - val_accuracy: 0.7486
Epoch 3/30
49/49 [==============================] - 2s 39ms/step - loss: 0.6612 - accuracy: 0.7532 - val_loss: 0.6478 - val_accuracy: 0.7593
Epoch 4/30
49/49 [==============================] - 2s 39ms/step - loss: 0.6222 - accuracy: 0.7840 - val_loss: 0.6034 - val_accuracy: 0.7871
Epoch 5/30
49/49 [==============================] - 2s 38ms/step - loss: 0.5666 - accuracy: 0.8112 - val_loss: 0.5487 - val_accuracy: 0.8070
Epoch 6/30
49/49 [==============================] - 2s 36ms/step - loss: 0.5052 - accuracy: 0.8344 - val_loss: 0.4946 - val_accuracy: 0.8284
Epoch 7/30
49/49 [==============================] - 2s 40ms/step - loss: 0.4482 - accuracy: 0.8555 - val_loss: 0.4478 - val_accuracy: 0.8394
Epoch 8/30
49

In [12]:
index=np.random.randint(1,1000)
print(index)
user_review=test_reviews.loc[index]
print(user_review)

122
Reviews      <START now infamous western that was at its ti...
Sentiment                                             negative
Name: 122, dtype: object


In [13]:
user_review=test_data[index]
user_review=np.array([user_review])
if (model.predict(user_review)>0.5).astype("int32"):
  print("positive sentiment")
else:
  print("negative sentiment")

1/1 [==============================] - 0s 280ms/step
negative sentiment
